# Phase 3: Trading Strategy Development

This notebook demonstrates the implementation of the Hybrid AI Trading Strategy, including:
1. Market Regime Detection
2. Volatility-Based Position Sizing
3. Backtesting with Backtrader
4. Risk Management Integration

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Add root directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.strategies.regime_detector import MarketRegimeDetector
from src.strategies.hybrid_ai_strategy import HybridAIStrategy
from src.backtesting.backtest_engine import BacktestEngine
from src.risk.risk_manager import RiskManager

## 1. Prepare Market Data

In [ ]:
def generate_backtest_data(days=365):
    np.random.seed(42)
    dates = pd.date_range(start='2023-01-01', periods=days, freq='D')
    
    # Generate trending price data with some noise
    returns = np.random.normal(0.0005, 0.02, days)
    # Add some regime shifts
    returns[100:150] -= 0.01  # Bearish period
    returns[250:300] += 0.01  # Strong bullish
    
    prices = 100 * np.exp(np.cumsum(returns))
    
    df = pd.DataFrame({
        'open': prices * (1 + np.random.normal(0, 0.002, days)),
        'high': prices * (1 + abs(np.random.normal(0.005, 0.005, days))),
        'low': prices * (1 - abs(np.random.normal(0.005, 0.005, days))),
        'close': prices,
        'volume': np.random.uniform(5000, 20000, days)
    }, index=dates)
    
    return df

df = generate_backtest_data()
df['close'].plot(figsize=(12, 6), title='Mock SOL Price History')
plt.show()

## 2. Market Regime Analysis

In [ ]:
detector = MarketRegimeDetector(window=20)
returns = df['close'].pct_change().dropna()
regime = detector.calculate_regime(returns)

detector.plot_regime_analysis(returns, regime)

## 3. Run Strategy Backtest

In [ ]:
# Mock components for Strategy
class MockModel:
    def predict(self, X):
        # Randomly predict 1 (long) or 0 (hold)
        pred = np.random.choice([0, 1])
        prob = 0.6 if pred == 1 else 0.4
        return pred, prob

class MockFE:
    feature_columns = ['f1', 'f2']

engine = BacktestEngine(initial_cash=100000)
engine.setup_cerebro()
engine.add_data(df)
engine.add_strategy(HybridAIStrategy, 
                   model=MockModel(), 
                   feature_engineer=MockFE(),
                   enable_regime_filter=True)

engine.run_backtest()
metrics = engine.get_performance_metrics()

## 4. Performance Metrics

In [ ]:
metrics_df = pd.DataFrame([metrics]).T
metrics_df.columns = ['Metric Value']
display(metrics_df)